In [ ]:
!pip install fastapi uvicorn[standard] ollama nest_asyncio
!pip install uvicorn[standard]
!pip install requests



In [4]:
import sys
!{sys.executable} -m pip install requests


  Using cached requests-2.32.4-py3-none-any.whl (64 kB)
  Using cached charset_normalizer-3.4.2-cp311-cp311-win_amd64.whl (105 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl (129 kB)



[notice] A new release of pip available: 22.3 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
from fastapi.middleware.cors import CORSMiddleware
import requests
import nest_asyncio
import uvicorn

nest_asyncio.apply()

app = FastAPI()

# Configurar CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Permite qualquer origem. Troque por seu frontend se quiser restringir.
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Modelo da requisição
class TraducaoRequest(BaseModel):
    texto: str

@app.post("/traduzir")
def traduzir(req: TraducaoRequest):
    prompt = f"Traduza o seguinte texto do inglês para o português:\n\n\"{req.texto}\""

    try:
        resposta = requests.post(
    "http://localhost:11434/api/chat",
    json={
        "model": "gemma3",
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "stream": False
    },
)


        data = resposta.json()
        return {"traducao": data.get("message", {}).get("content", "Erro ao gerar tradução.")}
    except Exception as e:
        return {"traducao": f"Erro ao conectar com o Ollama: {str(e)}"}

# Execução local
if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)


INFO:     Started server process [28452]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:56681 - "POST /traduzir HTTP/1.1" 200 OK
